In [29]:
import pandas as pd
import numpy as np
#we want to give highest weightage 50 
#second is 30 
#third is 20
import math

In [30]:
path = '/Users/yutingmei/APlusBernstein-Project/data/pred/lstm_preds.csv'
target = pd.read_csv(path)
# target = target.iloc[1: , :]
# target = target.reset_index(drop=True)

In [31]:
target

,sc,mom,value,.pred_prob_sc_lstm,.pred_prob_mom_lstm,.pred_prob_value_lstm,.pred_sc_lstm,.pred_mom_lstm,.pred_value_lstm,s_&_p_500_ret,sc_1d_fwd_ret,mom_1d_fwd_ret,value_1d_fwd_ret
0,1,0,1,0.495916,0.551642,0.551642,0,1,0,NaN,0.005083,-0.000662,0.001758
1,0,0,1,0.507404,0.537511,0.537511,1,1,0,-0.000333,-0.008109,-0.002763,-0.001990
2,1,1,1,0.513883,0.548627,0.548627,1,1,0,-0.002512,0.008351,0.009081,0.006174
3,1,1,1,0.525089,0.551225,0.551225,1,1,0,0.006082,0.001307,0.007427,0.001928
4,0,1,1,0.535904,0.556041,0.556041,1,1,0,-0.000212,0.000261,0.001359,0.002828
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1841,1,0,0,0.473095,0.508067,0.508067,0,1,0,-0.000215,0.001707,-0.004040,-0.002439
1842,0,0,0,0.475162,0.510649,0.510649,0,1,0,-0.000846,-0.003714,-0.004661,0.006674
1843,0,0,0,0.484604,0.517438,0.517438,0,1,0,0.006763,-0.013597,-0.007605,-0.009775
1844,1,0,1,0.492156,0.528105,0.528105,0,1,0,-0.007195,0.004891,-0.007206,0.008705


In [37]:
def long_short_index(target):
    pred = target.iloc[:,6:9]
    long_index = []
    short_index = []
    for i in range(target.shape[0]):
        if pred.iloc[i,:].sum() >= 2:
            long_index.append(i)
        else:
            short_index.append(i)
    return long_index, short_index

In [49]:
def judge_weight(pred_prob):
    init_weight = [0 * i for i in range(3)]
    weight_sum = []
    for i in range(pred_prob.shape[0]):
        row = list(pred_prob.iloc[i,:])
        max_value = max(row)
        index_max = [index for index in range(len(row)) if row[index] == max_value]
        if len(index_max) == 1:
            index_max = row.index(max(row))
            index_min = row.index(min(row))
            init_weight[index_max] = 0.5
            init_weight[index_min] = 0.2
            index_med = init_weight.index(0)
            init_weight[index_med] = 0.3
        elif len(index_max) == 2:
            index_min = row.index(min(row))
            init_weight[index_min] = 0.2
            init_weight = list(map(lambda x: 0.4 if x == 0 else x, init_weight))
        else:
            init_weight = [1/3, 1/3, 1/3]
        weight_sum.append(init_weight)
        init_weight = [0 * i for i in range(3)]
    return weight_sum  

In [50]:
def weighted_long_short_etf(target):
    start = 100000
    s_p = target.iloc[:, -4]#s&p
    #s_p = target["s_&_p_500_ret"]
    # real = target.iloc[:, 0:3]#real values in 1 or 0
    excess_return = []
    pred = target.iloc[:,6:9]
    op = target.iloc[:,-3:]
    # total_list = []
    return_list = []
    weight = pd.DataFrame(judge_weight(target.iloc[:, 3:6]))
    total_list = []
    for i in range(target.shape[0]):
        row = op.iloc[i, :]
        direct_pred = list(map(lambda x: -1 if x == 0 else 1, pred.iloc[i,:]))
        weight_row = weight.iloc[i,:]
        add = sum(list(map(lambda x,y,z: (start * z) * (1 +  y * x), row, direct_pred, weight_row)))
        total_list.append(add)
        # if pred.iloc[i,:].sum() >= 2:
        #     # pred direction
        #     op_invest = 1
        #     op_s_p = -1
        # else:
        #     op_invest = -1
        #     op_s_p = 1
        # add = sum(list(map(lambda x: (start / 3) * (1 + op_invest * x), row)))
        # total = (add) + ((1 + (op_s_p * s_p[i])) * start)
        # total_list.append(total)
        percentage = (add - start) / (start) 
        return_list.append(percentage)
        if i == target.shape[0] - 1:
            excess_return.append(None)
        else:
            excess_return.append(percentage - s_p[i + 1])
        start = add 
    return excess_return, return_list, total_list

In [51]:
excess_return, return_list, total = weighted_long_short_etf(target)

In [52]:
return_list[:10]

[-0.0019842731851787538,
 -0.0019308007458343794,
 0.002832943415771283,
 0.002460751157041066,
 -0.0005355915976394047,
 0.002287089658873039,
 -0.005183992180046651,
 0.0036303889046347,
 -0.00038458358743792786,
 0.0026112988693071386]

In [53]:
excess_return[:10]

[-0.0016513001177871538,
 0.0005809800521473208,
 -0.0032488293306611174,
 0.002672952171253066,
 -0.0008838928128841047,
 -1.9602468451861144e-05,
 0.0073922004414136485,
 -0.007187553047871999,
 -0.005550771701942328,
 0.003958424460365139]

In [54]:
total[0]

99801.57268148212

In [55]:
total[-1]

99313.95460230873

In [45]:
#here we need to add weights instead of multiplying by 1/3 and those weighst are calculated by the probability in the table
#for highest probability give weight 50 
#for 2nd highest give 30 
#for lowest give 20

# def balanced_profolio(target):
#     #target_ = target.iloc[:, 1:4]
#     # target = target.iloc[:, -3:]#the return values
#     #s_p = target.iloc[:, 4]
#     s_p = target.iloc[:, -4]#s&p
#     #s_p = target["s_&_p_500_ret"]
#     real = target.iloc[:, 0:3]#real values in 1 or 0
#     compare_result = []
#     pred = target.iloc[:,6:9]
#     op = target.iloc[:,-3:]
#     weight = pd.DataFrame(judge_weight(target.iloc[:, 3:6]))
#     total_list = []
#     start = 20000
#     for i in range(target.shape[0]):
#         row = op.iloc[i, :]
#         if pred.iloc[i,:].sum() >= 2:
#             # pred direction
#             op_invest = 1
#             op_s_p = -1
#             weight_start = 0.7
#             weight_start_s_p = 0.3
#         else:
#             op_invest = -1
#             op_s_p = 1
#             weight_start = 0.3
#             weight_start_s_p = 0.7
#         weight_row = weight.iloc[i,:]
#         add = sum(list(map(lambda x,y: (start * weight_start * y) * (1 + op_invest * x), row, weight_row)))
#         total = (add) + ((1 + (op_s_p * s_p[i])) * start * weight_start_s_p)
#         total_list.append(total)
#         compare_result.append((total - (start * weight_start + start * weight_start_s_p)) / (start * weight_start + start * weight_start_s_p))
#         start = total
#     return compare_result, total_list